In [91]:
# Webscraping
import requests
import re
from bs4 import BeautifulSoup

# Analysis
import numpy as np
import pandas as pd

In [3]:
url = 'https://en.wikipedia.org/wiki/1999_term_opinions_of_the_Supreme_Court_of_the_United_States'

In [4]:
website_url = requests.get(url).text

In [5]:
soup = BeautifulSoup(website_url, 'lxml')

In [9]:
table = soup.find('table', {'class': 'wikitable sortable'})

In [25]:
case_links = table.find_all('a')
cases = [ case.get('title') for case in case_links ]

In [41]:
headers = table.find_all('th')

In [45]:
justices = [ header.text.strip() for header in headers ][4:13]
print(justices)

['Rehnquist', 'Stevens', "O'Connor", 'Scalia', 'Kennedy', 'Souter', 'Thomas', 'Ginsburg', 'Breyer']


In [132]:
first_row = table.find('tr')
rows = first_row.next_siblings

j1 = []
j2 = []
j3 = []
j4 = []
j5 = []
j6 = []
j7 = []
j8 = []
j9 = []
j_votes = []
cases = []

for i, row in enumerate(rows):
    if i % 2 == 0:
        votes = []
        data = row.find_all('td')
        for m, datum in enumerate(data):
            if m == 1:
                cases.append(datum.text.strip())
            elif m in [0, 2, 3]:
                continue
            else:
                if re.match(r'padding', datum['style']):
                    vote = datum['data-sort-value']
                    if re.match(r'^(.*)?(?=<)', datum['data-sort-value']):
                        vote = re.match(r'^(.*)?(?=<)', datum['data-sort-value'])[0]
                    votes.append(vote)
        if votes != []:
            j_votes.append(votes)

In [134]:
len(j_votes)

85

In [52]:
case_rows = [ row for row in rows ]

In [53]:
case_rows

[]